In [1]:
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
class splitData():
    def __init__(self):
        self.X_train = []
        self.X_test = []
        self.y_train = []
        self.y_test = []
    
    def split(self, x, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [16]:
features = ['district_encoder',\
        'area',\
        'new_num_floors',\
        'new_bedrooms',\
        'houseTypes_Bán Luxury home',\
        'houseTypes_Bán Nhà',\
        'houseTypes_Bán Nhà cổ',\
        'houseTypes_Bán Nhà mặt phố',\
        'houseTypes_Bán Nhà riêng']

df = pd.read_excel('HCM_data.xlsx')
df_tranform = pd.DataFrame(data = StandardScaler().fit_transform(df.loc[:, features].values), columns = features)
y = df['price'].values
x = df_tranform[features].values
data = splitData()
data.split(x, y)

# RNN seq

## simple ANN


In [17]:

class ModelRNN(keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = keras.layers.Dense(20, activation='relu') #fully connected networks
        self.dense2 = keras.layers.Dense(20, activation='relu')
        self.dense3 = keras.layers.Dense(20, activation='relu')
        self.dense4 = keras.layers.Dense(1, activation= 'relu')
        
        self.model = Sequential([self.dense1, self.dense2, self.dense3, self.dense4])
        
    def call(self,inputs):
        return self.model(inputs)
    
    def compile(self, optimizer="adam", loss="mse"):
        super().compile(optimizer, loss)
        
    def fit(self, data, epochs = 500, batch_size=128):
        super().fit(data.X_train, data.y_train, epochs, batch_size)
        
    def evaluation(self, data):
        # Predict the values for the test dataset.
        y_pred = self.predict(data.X_test)

        # Calculate the MSE, MAE, and explained variance score.
        from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score
        mse = mean_squared_error(data.y_test, y_pred)
        mae = mean_absolute_error(data.y_test, y_pred)
        evs = explained_variance_score(data.y_test, y_pred)

        # Print the evaluation metrics.
        print("MSE: ", mse)
        print("MAE: ", mae)
        print("variance: ", evs)

In [18]:
model_rnn = ModelRNN()
model_rnn.compile(optimizer='adam', loss='mse')
model_rnn.fit(data, epochs=500, batch_size =128)
model_rnn.evaluation(data)

Epoch 1/128
9/9 [==============================] - 0s 1ms/step - loss: 534.5526
Epoch 2/128
9/9 [==============================] - 0s 990us/step - loss: 530.4365
Epoch 3/128
9/9 [==============================] - 0s 947us/step - loss: 525.9086
Epoch 4/128
9/9 [==============================] - 0s 1ms/step - loss: 520.4193
Epoch 5/128
9/9 [==============================] - 0s 830us/step - loss: 513.5623
Epoch 6/128
9/9 [==============================] - 0s 1ms/step - loss: 505.5222
Epoch 7/128
9/9 [==============================] - 0s 1ms/step - loss: 495.8611
Epoch 8/128
9/9 [==============================] - 0s 917us/step - loss: 483.2390
Epoch 9/128
9/9 [==============================] - 0s 1ms/step - loss: 469.7096
Epoch 10/128
9/9 [==============================] - 0s 1ms/step - loss: 454.9534
Epoch 11/128
9/9 [==============================] - 0s 1ms/step - loss: 442.5753
Epoch 12/128
9/9 [==============================] - 0s 984us/step - loss: 429.4141
Epoch 13/128
9/9 [=========

# seq2seq

In [47]:
class TransformerModel(keras.Model):
    
    def __init__(self):
        super().__init__()

        self.encoder = Sequential([
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(64, activation='relu'),
        ])

        self.decoder = Sequential([
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(128, activation='relu')
        ])
        self.dense = Dense(1)
        
    def call(self, inputs):
        encoder_outputs = self.encoder(inputs)
        decoder_outputs = self.decoder(encoder_outputs)
        outputs = self.dense(decoder_outputs)
        return outputs
    
    def compile(self, optimizer='adam', loss='mse'):
        super().compile(optimizer, loss)

    def fit(self, x_train, y_train, epochs=1000, batch_size =128):
        super().fit(x_train, y_train, epochs, batch_size)
    
    def evaluation(self, data):
            # Predict the values for the test dataset.
        y_pred = self.predict(data.X_test)

        # Calculate the MSE, MAE, and explained variance score.
        from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score
        mse = mean_squared_error(data.y_test, y_pred)
        mae = mean_absolute_error(data.y_test, y_pred)
        evs = explained_variance_score(data.y_test, y_pred)

        # Print the evaluation metrics.
        print("MSE: ", mse)
        print("MAE: ", mae)
        print("variance: ", evs)
    




In [48]:
model = TransformerModel()
model.compile(optimizer='adam', loss='mse')
model.fit(data.X_train, data.y_train, epochs=500, batch_size =128)
model.evaluation(data)

Epoch 1/128
9/9 [==============================] - 0s 3ms/step - loss: 532.6818
Epoch 2/128
9/9 [==============================] - 0s 3ms/step - loss: 469.9922
Epoch 3/128
9/9 [==============================] - 0s 3ms/step - loss: 428.3555
Epoch 4/128
9/9 [==============================] - 0s 3ms/step - loss: 414.5823
Epoch 5/128
9/9 [==============================] - 0s 4ms/step - loss: 413.4904
Epoch 6/128
9/9 [==============================] - 0s 9ms/step - loss: 399.8412
Epoch 7/128
9/9 [==============================] - 0s 5ms/step - loss: 398.6252
Epoch 8/128
9/9 [==============================] - 0s 5ms/step - loss: 390.8647
Epoch 9/128
9/9 [==============================] - 0s 5ms/step - loss: 384.7948
Epoch 10/128
9/9 [==============================] - 0s 6ms/step - loss: 380.6484
Epoch 11/128
9/9 [==============================] - 0s 5ms/step - loss: 375.3599
Epoch 12/128
9/9 [==============================] - 0s 7ms/step - loss: 363.6794
Epoch 13/128
9/9 [===================

In [21]:

vocab_size = 256

class Transformer(keras.Model):

    def __init__(self, num_layers, d_model, num_heads, dropout_rate):
        super(Transformer, self).__init__()

        self.encoder = TransformerEncoder(num_layers, d_model, num_heads, dropout_rate)
        self.decoder = TransformerDecoder(num_layers, d_model, num_heads, dropout_rate)

    def call(self, inputs, training=False):
        encoder_outputs = self.encoder(inputs, training=training)
        decoder_outputs = self.decoder(encoder_outputs, training=training)

        return decoder_outputs

class TransformerEncoder(keras.layers.Layer):

    def __init__(self, num_layers, d_model, num_heads, dropout_rate):
        super(TransformerEncoder, self).__init__()

        self.attention_layers = []
        for _ in range(num_layers):
            self.attention_layers.append(
                keras.layers.MultiHeadAttention(
                    num_heads=num_heads,
                    d_model=d_model,
                    dropout_rate=dropout_rate
                )
            )
        self.feed_forward_layers = []
        for _ in range(num_layers):
            self.feed_forward_layers.append(
                keras.layers.Dense(
                    d_model,
                    activation="relu",
                    dropout_rate=dropout_rate
                )
            )

    def call(self, inputs, training=False):
        for attention_layer in self.attention_layers:
            inputs = attention_layer(inputs, inputs, training=training)
            inputs = keras.layers.LayerNormalization(epsilon=1e-6)(inputs)

        for feed_forward_layer in self.feed_forward_layers:
            inputs = feed_forward_layer(inputs)
            inputs = keras.layers.LayerNormalization(epsilon=1e-6)(inputs)

        return inputs

class TransformerDecoder(keras.layers.Layer):

    def __init__(self, num_layers, d_model, num_heads, dropout_rate):
        super(TransformerDecoder, self).__init__()

        self.attention_layers = []
        for _ in range(num_layers):
            self.attention_layers.append(
                keras.layers.MultiHeadAttention(
                    num_heads=num_heads,
                    d_model=d_model,
                    dropout_rate=dropout_rate
                )
            )
        self.feed_forward_layers = []
        for _ in range(num_layers):
            self.feed_forward_layers.append(
                keras.layers.Dense(
                    d_model,
                    activation="relu",
                    dropout_rate=dropout_rate
                )
            )

        self.final_layer = keras.layers.Dense(
            vocab_size,
            activation="softmax"
        )

    def call(self, inputs, encoder_outputs, training=False):
        for attention_layer in self.attention_layers:
            inputs = attention_layer(inputs, encoder_outputs, training=training)
            inputs = keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
        for feed_forward_layer in self.feed_forward_layers:
            inputs = feed_forward_layer(inputs)
            inputs = keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
        outputs = self.final_layer(inputs)
        return outputs
    
    def compile(self, )